In [1]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
from scipy import stats
import random
from math import floor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jafar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
q='''SELECT tweet, tweet_created_at FROM manager_world_tweets_ph2 
    Where 
    tweet_created_at like '%2019%';'''
cur.execute(q)
rows_mng = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_mng))

Database opened successfully
5983238


In [3]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
q='''SELECT tweet, tweet_created_at FROM ent_world_tweets_ph2 
    Where 
    tweet_created_at like '%2019%';'''
cur.execute(q)
rows_ent = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_ent))

Database opened successfully
5237437


In [4]:
for i, row in enumerate(rows_mng[0:10]):
    print(i,row)
    print(i,row[1].split('-'))

0 ('*Gasp* it makes noise (taking about phone)', '2019-08-21 22:55:51')
0 ['2019', '08', '21 22:55:51']
1 ('...okay BOOMER', '2019-11-10 03:49:52')
1 ['2019', '11', '10 03:49:52']
2 ('@ Emily https://t.co/W7791cTGNu', '2019-10-06 23:30:09')
2 ['2019', '10', '06 23:30:09']
3 ('@abbbbbbby2 Fr single buddies again', '2019-09-21 19:31:31')
3 ['2019', '09', '21 19:31:31']
4 ('@abbbbbbby2 WAHT WAS MY ADVICE', '2019-09-21 19:48:53')
4 ['2019', '09', '21 19:48:53']
5 ('@bean__boi__ @TheReal0J32 Yeppers here you go �', '2019-12-26 20:13:42')
5 ['2019', '12', '26 20:13:42']
6 ('How ima cook for him � https://t.co/4WnYhxjEmt', '2019-09-19 16:10:12')
6 ['2019', '09', '19 16:10:12']
7 ('@bean__boi__ The hippy one?', '2019-12-30 03:28:48')
7 ['2019', '12', '30 03:28:48']
8 ('@bean__boi__ Yea of course', '2019-12-30 03:32:03')
8 ['2019', '12', '30 03:32:03']
9 ('@bean__boi__ Yeah he is', '2019-12-30 03:29:36')
9 ['2019', '12', '30 03:29:36']


In [5]:
ind_mng_month = dict()

for i, row in enumerate(rows_mng):
    info=row[1].split('-')
    if str(info[1]) not in ind_mng_month:
        ind_mng_month[ str(info[1]) ]=[i] 
    else:
        ind_mng_month[ str(info[1]) ].append(i)
      

In [6]:
ind_ent_month = dict()

for i, row in enumerate(rows_ent):
    info=row[1].split('-')
    if str(info[1]) not in ind_ent_month:
        ind_ent_month[ str(info[1]) ]=[i] 
    else:
        ind_ent_month[ str(info[1]) ].append(i)
          

In [7]:
for i, row in enumerate(rows_mng[0:10]):
    print(i,row[1].split('-'))

0 ['2019', '08', '21 22:55:51']
1 ['2019', '11', '10 03:49:52']
2 ['2019', '10', '06 23:30:09']
3 ['2019', '09', '21 19:31:31']
4 ['2019', '09', '21 19:48:53']
5 ['2019', '12', '26 20:13:42']
6 ['2019', '09', '19 16:10:12']
7 ['2019', '12', '30 03:28:48']
8 ['2019', '12', '30 03:32:03']
9 ['2019', '12', '30 03:29:36']


In [35]:
type(ind_ent_month['01'])

list

In [11]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

In [12]:
def cleaning (text):
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)

    #removing some special charachter  
    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`;•«,@:~!=%&]+", ' ',text) # except ' ’
    
    #removing hashtag sign
    text=re.sub('#', '', text) 
    
    #removing numbers
    text=re.sub("(\d+)",' ',text)
#    print(text)
    #removing numbers (not attached)
#    text=re.sub(r"(^|\s)-?(\d+)\.?(\d*)(\s)", '', text)
#    text=re.sub("(^||\d+)\.(\d+) ",'',text)
#    text=re.sub("[0-9+]\.?(\d+)(\.)",' ',text)
#    text=re.sub("[0-9+]\.?(\d+)($)",' ',text)
#    print(text)
    
#    return text

#def lower_case (text):
    text = text.lower() 
#    print(text)
#    return text_lower

#def tokenization (text):
    text= nltk.word_tokenize(text)
#    print (text)
#    return text_tokens

#def removing_stopwords (text):
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()]
#    text = [word for word in text if word not in stopwords.words()]
    text = [word for word in text if word not in cachedStopWords]
#    print(text)
#    return text_without_sw

#def stemming (text):
    #stemmer = SnowballStemmer("english")
    #text_stems=stemmer.stem(tokens_without_sw)
#    text = [stemmer.stem(word) for word in text]
    text = [Stem(word) for word in text]

#    print(text)
    
    text=' '.join(text)
#    print(text)
#    print(text,'\n')   
    
    return text

In [13]:
mng_all_cleaned=[]
for tweet in rows_mng:
    mng_all_cleaned.append(cleaning(tweet[0]))

In [14]:
ent_all_cleaned=[]
for tweet in rows_ent:
    ent_all_cleaned.append(cleaning(tweet[0]))

In [37]:
ind_sorted=sorted(ind_ent_month.keys())
#ind_ent_month[ind_sorted[0]]
ind_sorted

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [62]:
ent_tweet_month=dict()
ent_tweet=[]

freq_most=[10,500,1000,10000,100000]
for n_most in freq_most:
    
    for month in ind_sorted:

        ent_tweet=[]
        for jj in ind_ent_month[month]:
            ent_tweet.append(ent_all_cleaned[jj])

        ent_words=[]
        for tweet in ent_tweet:
            ent_words += tweet.split() 
            
            

        mng_tweet=[]
        for jj in ind_mng_month[month]:
            mng_tweet.append(mng_all_cleaned[jj])

        mng_words=[]
        for tweet in mng_tweet:
            mng_words += tweet.split()    


        chi2=[]
        p_value=[]

        for kk in range(5):

            ent_size=floor(len(ent_words)/2)
            ent_words_random=random.sample(ent_words,ent_size)

            mng_size=floor(len(mng_words)/2)
            mng_words_random=random.sample(mng_words,mng_size)

            all_words=mng_words_random+ent_words_random


            d_mng = dict()
            for c in mng_words_random:
                if c not in d_mng:
                    d_mng[c] = 1
                else:
                    d_mng[c] = d_mng[c] + 1

            d_mng_new=dict(sorted(d_mng.items(), key=lambda x: x[1], reverse=True))
            key_list_mng=list(d_mng_new.keys())
            values_list_mng=list(d_mng_new.values())
    #    print(list(d_mng_new.keys())[0:10])
    #    print(list(d_mng_new.values())[0:10])        

            d_ent = dict()
            for c in ent_words_random:
                if c not in d_ent:
                    d_ent[c] = 1
                else:
                    d_ent[c] = d_ent[c] + 1

            d_ent_new=dict(sorted(d_ent.items(), key=lambda x: x[1], reverse=True))
            key_list_ent=list(d_ent_new.keys())
            values_list_ent=list(d_ent_new.values())
    #    print(list(d_ent_new.keys())[0:10])
    #    print(list(d_ent_new.values())[0:10])       

            d_all = dict()
            for c in all_words:
                if c not in d_all:
                    d_all[c] = 1
                else:
                    d_all[c] = d_all[c] + 1

            d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
            key_list_all=list(d_all_new.keys())
            values_list_all=list(d_all_new.values())
    #    print(list(d_all_new.keys())[0:10])
    #    print(list(d_all_new.values())[0:10])        

    #    print(len(key_list_all))
    #    print(len(key_list_mng))
    #    print(len(key_list_ent))



            frequent=key_list_all[0:n_most] # 
        #frequent

            text1_freq_values=[]
            text2_freq_values=[]

            for ii in frequent:
        #    print(ii)
        #    print(d_ent_new[ii])
        #    print(d_mng_new[ii])
                try: 
                    text1_freq_values.append(d_ent_new[ii])
                except:
                    text1_freq_values.append(0)
                try:    
                    text2_freq_values.append(d_mng_new[ii])
                except:
                    text2_freq_values.append(0)


    #        except:
    #            print(ii)
    #            print(d_mng_new[ii])
    #            print(d_ent_new[ii])

        #print(text1_freq_values )  
        #print(text2_freq_values)    

            chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
    #    print(chi2_stat)
    #    print(p_val)
            chi2.append(chi2_stat)
            p_value.append(p_val)

        CHI2_mean=np.mean(chi2)
        CHI2_std=np.std(chi2)
        P_VALUE_mean=np.mean(p_value)
        P_VALUE_std=np.std(p_value)

        
        print('For', n_most,' most frequent words for month',month,'we have')
        print('chi2:',chi2)
        print('CHI2_mean:',CHI2_mean)
        print('CHI2_std:',CHI2_std)
        print('p_value:',p_value)
        print('P_VALUE_mean:',P_VALUE_mean)
        print('P_VALUE_std:',P_VALUE_std)
        print('\n The 10 most frequnt words in ent random subcorpus are:')
        print(key_list_ent[0:10])
        print('\n The 10 most frequnt words in mng random subcorpus are:')
        print(key_list_mng[0:10])
        print('--'*50)
        print()

For 10  most frequent words for month 01 we have
chi2: [279.80086023495284, 290.95989283922665, 247.05807323623668, 258.15692718672545, 291.04597557587437]
CHI2_mean: 273.4043458146032
CHI2_std: 17.815440394694797
p_value: [4.984945611135677e-55, 2.155413230814179e-57, 4.168189766835939e-48, 1.8885275136168345e-50, 2.0667333150691025e-57]
P_VALUE_mean: 8.37415108937763e-49
P_VALUE_std: 1.6654033903284112e-48

 The 10 most frequnt words in ent random subcorpus are:
['rt', '’', "'s", 'like', "n't", 'get', 'amp', 'one', 'year', 'peopl']

 The 10 most frequnt words in mng random subcorpus are:
['rt', '’', "'s", 'get', 'like', 'one', "n't", 'year', 'new', 'go']
----------------------------------------------------------------------------------------------------

For 10  most frequent words for month 02 we have
chi2: [617.2624829139462, 416.2366801190726, 413.10740446277913, 611.6680635270656, 614.6631476901179]
CHI2_mean: 534.5875557425964
CHI2_std: 97.9316145120335
p_value: [4.1256010697822

It is seen that for all months the ent and mng corpuses are different.   